In [1]:
import torch 

n_targets = 5
n_samples = 10
n_features = 3
mt_explanation_per_sample = [
    tuple (
        torch.randn(1, 100) for _ in range(n_features)
    ) for _ in range(n_targets)
]

mt_explanations_btf = [mt_explanation_per_sample for _ in range(n_samples)]

In [2]:
mt_explanations_tbf = list(map(list, zip(*mt_explanations_btf, strict=True))) # perform outer transpose (B, T, F) -> (T, B, F)
mt_explanations_tfb = [tuple(map(torch.cat, zip(*target_explanations_bf, strict=True))) for target_explanations_bf in mt_explanations_tbf]  # perform inner transpose (B, F) -> (F, B)

for target_explanations_fb in mt_explanations_tfb:
    assert len(target_explanations_fb) == 3  # Should be n_features
    assert isinstance(target_explanations_fb, tuple)
    for feature_explanations_b in target_explanations_fb:
        assert feature_explanations_b.shape == (10, 100)  # Should be (n_samples, 1)

In [ ]:
from atria_insights.data_types._explanation_state import MultiTargetBatchExplanation, MultiTargetSampleExplanation

data =[MultiTargetSampleExplanation(value=mt_explanation_per_sample) for _ in range(n_samples)]
sample_mt_explanations = []
for d in data:
    assert isinstance(d, MultiTargetSampleExplanation), (
        "All explanations must be of type SampleMultiTargetExplanation in multi-target case."
    )
    sample_mt_explanations.append(d)
batch_mt_explanations = MultiTargetBatchExplanation.fromlist(
    sample_mt_explanations
)

In [4]:


mt_explanations = [d.explanations for d in data]
# explanations_list is now a list of lists of tuples
# first list is per sample
# second list is per target
# tuple is the explanation tensors for that target with each tensor in tuple having shape (1, ...)
# we need to transpose this into list of tuples of lists
# so that we can batch per target
transposed_mt_explanations = [
    tuple(map(list, zip(*mt_explanations_per_sample, strict=True)))
    for mt_explanations_per_sample in mt_explanations
]
print("transposed_explanations", transposed_mt_explanations)
total_samples = len(data)
total_targets = len(transposed_mt_explanations[0])
for sample_explanations in transposed_mt_explanations:
    assert len(sample_explanations) == total_targets, (
        "Each sample's explanations must match the total number of targets."
    )
    for explanation in sample_explanations:
        assert len(explanation) == total_samples, (
            "Each explanation list must match the total number of samples."
        )

AttributeError: 'SampleMultiTargetExplanation' object has no attribute 'explanations'